# Algoritmo di ricerca output massimo

Il programma cerca entro un numero massimo di tentativi (max_tries) la configurazione del sistema che massimizza un singolo output. 

## 1) Import

In [5]:
# Required imports
import sys
username = 'admin' # 'admin' for BO1 lab
sys.path.append('c:/users/'+ username +'/miniconda3/lib/site-packages')
path_to_MyCustomPackage = '../' # relative path to the MyCustomPackage folder 
sys.path.append(path_to_MyCustomPackage)
from MyCustomPackage import mycustommodule
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual, widgets
from matplotlib.patches import Polygon
from matplotlib.ticker import MaxNLocator
from cycler import cycler
from IPython.display import display
from IPython.display import clear_output
import qontrol
import os
import time
import numpy as np
import datetime
import os.path
import math

%matplotlib widget

## 2) Dichiarazione variabili e setup driver

In [6]:
# Variable declarations
max_tries = 2
voltage = 12.0
repetitions = 2
channels = [4,8,9,11,15,16,17,19,20,21,22,25,26,27] # the channels connected to the device
#channels = [k for k in range(10)] # the channels connected to the device
channel_PD = 3 # the channel connected to the photodiode
voltage_PD = 0 # the voltage to be set to the photodiode
voltage_start = [0.0 for j in range(len(channels))] # starting voltage values for each channel
voltage_stop = [voltage for j in range(len(channels))] # stopping voltage values for each channel
voltage_step = [0.1 for j in range(len(channels))] # voltage steps for each channel
max_current = 10 # current compliance
max_voltage = 12.0 # voltage compliance
measured_voltage = [[[] for i in range(len(channels))] for j in range(len(channels))] # measured_voltage[channels][channel_under_sweep][measurement]
measured_current = [[[] for i in range(len(channels))] for j in range(len(channels))] # current measurements
PD_voltage = [[] for i in range(len(channels))] # PD voltage measurements
PD_current = [[] for i in range(len(channels))] # PD current measurements
F_move_on = 0 # FLAG : 0 for pausing when a channel sweep is done
F_overlapping_plots = 1 # FLAG : 0 for non overlapping plots, not 0 otherwise 
F_triangular = 1 # FLAG : 0 for ramp sweep, 1 for triangular sweep
now = datetime.datetime.now() # retrieve current date and time (for file name)

# Configuration (Laser and Amplifier)
laser_wavelength = 850
laser_power = 4.93
amplifier_gain = 40

device = "Q3_+02_+08_" + str(voltage)  + "V_2sweep"

# Set the right path and file
save_path = "G:/Shared drives/FMPS Archive/Projects/0_QPIXPAD/QPX4/Opt.Meas/Q3_+02_+08/"
file_name = "Single_channel_control_" + now.strftime("%d_%m_%Y_%H_%M_%S")+ device + '.txt' 
completeName = os.path.join(save_path, file_name)

# Build the header of the file as timestamp+configuration+annotation+voltage_parameters+description
timestamp = '%'+now.strftime("%d/%m/%Y %H:%M:%S")
configuration = '\n%Laser Wavelength:' + str(laser_wavelength) + '[nm]\n%Laser Power:' +str(laser_power) + '[mW]\n%Amplifier Gain:' \
              + str(amplifier_gain) + '[dB]'
# LEFT          RIGHT
# 1              1
# 2              2
# Le misure con annotazione (%Di oggi 20 gennaio da questa in poi sono trans, le altre precedenti erano CIS) sono 1 - 2
# Le misure con annotazione (%Min Scattering Trans) sono 1 - 1
annotation = '\n%TEST CON UN TERMISTORE SOLO\n'

voltage_parameters = '%' # voltage_parameters are voltage_start, voltage_stop, voltage_step, triangular
for element in voltage_start:
    voltage_parameters += str(element) + '\t'
voltage_parameters += '\n%'
for element in voltage_stop:
    voltage_parameters += str(element) + '\t'
voltage_parameters += '\n%'
for element in voltage_step:
    voltage_parameters += str(element) + '\t'
voltage_parameters += '\n%' + str(F_triangular) + '\n'

description = '%Swipe sulla tensione con onda sinusoidale su un canale, canali rimanenti a 0 V.Ciascun canale pilota una termoresistenza.' + device
header = timestamp+configuration+annotation+voltage_parameters+description

# Write the header and the column headers to the file
meas_file = open(completeName, "a")
meas_file.write(header)
meas_file.write("\n%PD at channel " + str(channel_PD) + ": Voltage [V]\tCurrent [mA]")
for j in range(len(channels)):
    meas_file.write("\tChannel " +str(channels[j]) + " : Voltage [V]\tCurrent [mA]")
meas_file.write("\tTimestamp")
meas_file.write("\n")


# Setup Qontroller
serial_port_name = "COM3" #"COM3" #"/dev/tty.usbserial-FT06QAZ5" # name of the USB Serial Port #//./COM11
q = qontrol.QXOutput(serial_port_name = serial_port_name, response_timeout = 0.1)

# Set the compliances
for channel in range(q.n_chs):
    q.imax[channel] = max_current
    q.vmax[channel] = max_voltage

# Video Debugging
print ("Qontroller '{:}' initialised with firmware {:} and {:} channels".format(q.device_id, q.firmware, q.n_chs) )

RuntimeError: Qontroller.__init__: Error: Unable to communicate with device on port COM3 (received response , "").

## 3) Acquisizione e visualizzazione dati

In [ ]:
# Starting Pico with range of 2V, sample interval of 250 microseconds and 5000 total samples

channel_range = ps.PS4000_RANGE['PS4000_2V']
sampleInterval = ctypes.c_int32(250) # in sample units specified by sampleUnits
sampleUnits = ps.PS4000_TIME_UNITS['PS4000_US']
sizeOfOneBuffer = 500 # size of a single buffer
numBuffersToCapture = 10 # totalSamples = sizeOfOneBuffer * numBuffersToCapture
pico_start(channel_range, sampleInterval = sampleInterval, sampleUnits = sampleUnits, sizeOfOneBuffer = sizeOfOneBuffer, numBuffersToCapture = numBuffersToCapture)

# Display status returns
logging.info(status)

target_configuration = [[0.0, [0.0 for j in range(len(channels))]] for i in range(len(channels))] # list of sublist of two elements. First element is output voltage, second element is a list of all driving voltages for each channel
max_attuale = 0.0

for i in range(max_tries): # ricerca fino a max_tries e proponi il miglior risultato
    #max_attuale = 0.0
    #sweep di tensione da su ogni canale
    for channel_n in range(len(channels)): # esamina tutti i dof (canali/termoresistenze)
        x = channel_n
        print("\nDRIVING CHANNEL {:} NOW \n" .format(channels[channel_n]))

        # set all channels to their voltage_start values
        for k in range(len(channels)):
            q.v[channels[k]] = float(voltage_start[k])

        #check if we want to generate a triangular sweep or not
        if F_triangular == 1:
            sweep_range = np.concatenate((np.arange(voltage_start[channel_n], voltage_stop[channel_n], voltage_step[channel_n]),\
                            np.arange(voltage_stop[channel_n], voltage_start[channel_n], -voltage_step[channel_n])))
        elif F_triangular == 0:
            sweep_range = np.arange(voltage_start[0], voltage_stop[0]+voltage_step[0], voltage_step[0])
        else:
            print('invalid value for triangular')
            
        for voltage_sweep in sweep_range: 
            q.v[channels[channel_n]] = float(voltage_sweep)
            temp_voltage = q.v[channels[channel_n]]

            valore_output_pico = pico_acquire_measurement(channel_range, sampleInterval = sampleInterval, sampleUnits = sampleUnits, sizeOfOneBuffer = sizeOfOneBuffer, numBuffersToCapture = numBuffersToCapture));
            
            if valore_output_pico > max_attuale: # se trovi un nuovo massimo, salvalo al posto del massimo finora trovato
                voltage_max = q.v[channels[channel_n]]
                max_attuale = valore_output_pico
            
        q.v[channel_n] = voltage_max # setta il canale attuale alla tensione corrispondente al suo massimo
        
        target_configuration[channel_n][0] = max_attuale # salva l'output del massimo per il canale attuale
        
        for k in range(len(channels)):
            target_configuration[channel_n][1][k] = q.v[channels[k]] # salva per il canale attuale, nella lista delle driving voltages, ciascun driving voltage
        
        """
        # at the end of each sweep, set all channels to their voltage_start values
        for k in range(len(channels)):
            q.v[channels[k]] = float(voltage_start[k])
        """
        
# Set all channels to zero
q.v[:] = 0
q.i[:] = 0

# Close the communication with the driver
q.close()

# Close the Picoscope
pico_stop()

In [ ]:
%matplotlib widget                                       
axis_color = 'lightgoldenrodyellow'

hf_general = plt.figure()
hf_general.set_size_inches(15, 8, forward=True)
ha_general = hf_general.add_subplot(121)      # Output intensities (simulated)

# Adjust the subplots region to leave some space for the sliders and buttons
hf_general.subplots_adjust(bottom=0.30)
ha_general.set_title('Channel Voltage vs  Channel Current ')
ha_general.set_xlabel('Voltage [V]')
ha_general.set_ylabel('Current [mA]')
ha_general.set_xlim([-0.1, max(voltage_stop)])
ha_general.set_ylim([-0.1, 20])
ha_general.grid()
plt.rc('axes', prop_cycle=(cycler('color', ['r', 'g', 'b', 'c', 'm', 'y', 'k'])))



# Draw the initial plot(s) and text(s)
# The 'line' variable is used for modifying the line later
lines = []
z = 0

for x in range(7):
    if x == 4:
        z = 1
    [line] = ha_general.plot(0, 0, marker='o', label='T' + str(20-x-z) + 'T16 - CH' + str(3+x))
    lines.append(line)

ha_general.legend()  
    
locator = MaxNLocator(nbins=6)

F_overlapping_plots = 1

"""
Channel     Resistance
3           T20T16
4           T19T16
5           T18T16
6           T17T16
7           T15T16
8           T14T16
9           T13T16
"""

T13T16_slider = widgets.FloatSlider(
    value=1, 
    min=0, max=voltage, step=0.1,
    description='T13T16 - CH8',
    continuous_update=True)  

T14T16_slider = widgets.FloatSlider(
    value=1, 
    min=0, max=voltage, step=0.1,
    description='T14T16 - CH8',
    continuous_update=True) 

T15T16_slider = widgets.FloatSlider(
    value=1, 
    min=0, max=voltage, step=0.1,
    description='T15T16 - CH8',
    continuous_update=True) 

T17T16_slider = widgets.FloatSlider(
    value=1, 
    min=0, max=voltage, step=0.1,
    description='CHANNEL 6, MZI',
    continuous_update=True) 

T18T16_slider = widgets.FloatSlider(
    value=1, 
    min=0, max=voltage, step=0.1,
    description='T18T16 - CH8',
    continuous_update=True) 

T19T16_slider = widgets.FloatSlider(
    value=1, 
    min=0, max=voltage, step=0.1,
    description='T19T16 - CH8',
    continuous_update=True) 

T20T16_slider = widgets.FloatSlider(
    value=1, 
    min=0, max=voltage, step=0.1,
    description='T20T16 - CH8',
    continuous_update=True) 

def plot(T13T16_slider, T14T16_slider, T15T16_slider, T17T16_slider, T18T16_slider, T19T16_slider, T20T16_slider):
    
    q.v[3] = float(T20T16_slider)
    q.v[4] = float(T19T16_slider)
    q.v[5] = float(T18T16_slider)
    q.v[6] = float(T17T16_slider)
    q.v[7] = float(T15T16_slider)
    q.v[8] = float(T14T16_slider)
    q.v[9] = float(T13T16_slider)
    
    for x in range(7):
        temp_voltage = q.v[x+3]
        temp_current = q.i[x+3]
        lines[x].set_xdata(temp_voltage)
        lines[x].set_ydata(temp_current)
    hf_general.canvas.draw() 

z = interactive(plot, T13T16_slider=T13T16_slider, T14T16_slider=T14T16_slider, T15T16_slider=T15T16_slider, T17T16_slider=T17T16_slider, \
               T18T16_slider=T18T16_slider, T19T16_slider=T19T16_slider, T20T16_slider=T20T16_slider)
display(z)

plt.show()